# ResNet50 model

## Import modules and load data from pickles

In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import pickle

In [2]:
data_path = os.path.join(os.getcwd(), '..', 'input')

In [3]:
height = 256
width = 256

In [4]:
with open(os.path.join(data_path, 'train_images_256x256.pkl'), 'rb') as fin:
    train_images = pickle.load(fin)
with open(os.path.join(data_path, 'train_responses.pkl'), 'rb') as fin:
    train_responses = pickle.load(fin)
with open(os.path.join(data_path, 'augmented_images_256x256.pkl'), 'rb') as fin:
    augmented_images = pickle.load(fin)
with open(os.path.join(data_path, 'augmented_responses.pkl'), 'rb') as fin:
    augmented_responses = pickle.load(fin)

In [5]:
images = np.concatenate([train_images, augmented_images], axis=0)
responses = np.concatenate([train_responses, augmented_responses], axis=0)
del train_images
del augmented_images
del train_responses
del augmented_responses

In [6]:
# Shuffle data
permutation = np.random.permutation(images.shape[0])
images = images[permutation, :, :, :]
responses = responses[permutation, :]

## Normalize images

In [7]:
# images = images / 255.
images = images * 2. / 255. - 1.

## Define model architecture

In [8]:
from keras import backend as K
from keras.applications.resnet50 import ResNet50
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [9]:
class roc_callback(Callback):
    """Define a callback which returns train ROC AUC after each epoch."""

    def __init__(self, training_data, validation_data=None):
        self.x = training_data[0]
        self.y = training_data[1]
        # self.x_val = validation_data[0]
        # self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        # y_pred_val = self.model.predict(self.x_val)
        # roc_val = roc_auc_score(self.y_val, y_pred_val)
        # print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        print('\rroc-auc: {}'.format(round(roc, 5)), end=80 * ' ' + '\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [10]:
def resnet50():
    resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(height, width, 3), pooling='max')
    last = resnet.output
    # x = Flatten()(last)
    x = Dropout(0.75)(last)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.75)(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs=[resnet.input], outputs=[x])

model = resnet50()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
model.fit(images, responses, batch_size=16, epochs=10, callbacks=[roc_callback(training_data=(images, responses))])

Epoch 1/10
27804/27804 [==============================] - 222s 8ms/step - loss: 0.8312 - acc: 0.8775
roc-auc: 0.99359                                                                                
Epoch 2/10
27804/27804 [==============================] - 207s 7ms/step - loss: 0.1903 - acc: 0.9381
roc-auc: 0.99644                                                                                
Epoch 3/10
27804/27804 [==============================] - 204s 7ms/step - loss: 0.1304 - acc: 0.9582
roc-auc: 0.99739                                                                                
Epoch 4/10
27804/27804 [==============================] - 206s 7ms/step - loss: 0.1030 - acc: 0.9660
roc-auc: 0.99848                                                                                
Epoch 5/10
27804/27804 [==============================] - 205s 7ms/step - loss: 0.0838 - acc: 0.9740
roc-auc: 0.99903                                                                                
Epoch 6/10

In [12]:
model.save('resnet50_10epoch.h5')

In [13]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-5, momentum=0.9), metrics=['accuracy'])
model.fit(images, responses, batch_size=16, epochs=10, callbacks=[roc_callback(training_data=(images, responses))])

Epoch 1/10
27804/27804 [==============================] - 211s 8ms/step - loss: 0.0364 - acc: 0.9893
roc-auc: 0.99981                                                                                
Epoch 2/10
27804/27804 [==============================] - 206s 7ms/step - loss: 0.0330 - acc: 0.9894
roc-auc: 0.99981                                                                                
Epoch 3/10
27804/27804 [==============================] - 204s 7ms/step - loss: 0.0339 - acc: 0.9899
roc-auc: 0.99982                                                                                
Epoch 4/10
27804/27804 [==============================] - 209s 8ms/step - loss: 0.0298 - acc: 0.9906
roc-auc: 0.99983                                                                                
Epoch 5/10
27804/27804 [==============================] - 206s 7ms/step - loss: 0.0322 - acc: 0.9899
roc-auc: 0.99984                                                                                
Epoch 6/10

In [14]:
model.save('resnet50_20epoch.h5')

In [15]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-6, momentum=0.9), metrics=['accuracy'])
model.fit(images, responses, batch_size=16, epochs=5, callbacks=[roc_callback(training_data=(images, responses))])

Epoch 1/5
27804/27804 [==============================] - 213s 8ms/step - loss: 0.0275 - acc: 0.9912
roc-auc: 0.99987                                                                                
Epoch 2/5
27804/27804 [==============================] - 206s 7ms/step - loss: 0.0329 - acc: 0.9914
roc-auc: 0.99988                                                                                
Epoch 3/5
27804/27804 [==============================] - 209s 8ms/step - loss: 0.0270 - acc: 0.9917
roc-auc: 0.99987                                                                                
Epoch 4/5
27804/27804 [==============================] - 208s 7ms/step - loss: 0.0297 - acc: 0.9913
roc-auc: 0.99988                                                                                
Epoch 5/5
27804/27804 [==============================] - 209s 8ms/step - loss: 0.0305 - acc: 0.9915
roc-auc: 0.99987                                                                                


In [16]:
model.save('resnet50_25epoch.h5')

## Load test set and predict

In [17]:
def img_as_array(image_id, size=None, image_set='train_images'):
    image_path = os.path.join(data_path, image_set, image_id)
    img = cv2.imread(str(image_path))
    if size is None:
        return img
    return cv2.resize(img, size)

In [18]:
test_dir = 'leaderboard_test_data'
holdout_dir = 'leaderboard_holdout_data'

In [19]:
test_images = []
test_ids = []
for image_id in os.listdir(os.path.join(data_path, test_dir)):
    img = img_as_array(image_id, image_set=test_dir)
    test_images.append(img.reshape(1, height, width, 3))
    test_ids.append(image_id)
for image_id in os.listdir(os.path.join(data_path, holdout_dir)):
    img = img_as_array(image_id, image_set=holdout_dir)
    test_images.append(img.reshape(1, height, width, 3))
    test_ids.append(image_id)
test_images = np.concatenate(test_images, axis=0)

In [20]:
# test_images = test_images / 255.
test_images = test_images * 2. / 255. - 1.

In [21]:
predictions = model.predict(test_images)

In [22]:
predictions = predictions.squeeze().tolist()

In [23]:
with open('submission.csv','w') as fout:
    fout.write("image_id,has_oilpalm\n")
    for image_id, has_oilpalm in zip(test_ids, predictions):
        fout.write("{},{}\n".format(image_id, has_oilpalm))